# Tree base model

## Decision Tree(CART : Classification and Regression Tree)
> **`Decision Tree`** 모델은 **예측/분류가 모두 가능**한 **지도학습** 머신러닝 모델이다.   
스무고개 게임을 하듯 여러 개의 가정을 데이터에 반영하고 이를 바탕으로 결정경계(decision boundary)를 생성  
모델 예측 및 분류 결과에 따른 해석이 굉장히 용이하여 **모델 해석이 필요한 문제에 사용**한다.ex)신용평가, 모델분류  
최근에는 `Decision Tree`모델을 베이스로 한 부스팅 트리 모델(**`Xgboost`**, **`LightGBM`**, **`Catboost`**)등으로 데이터분석 대회 수상을 하면서 실무 적용 케이스가 많아졌다.

### 모델구조
> 뿌리 노드(root node) : 최상위 노드, 모든 샘플 포함  
잎 노드(leaf node) : 최하위 노드, 여기에 속한 샘플이 어떤 클래스인지 결정 됨  
노드(node) : 뿌리 노드와 잎 노드 사이에 있는 노드  
가지(branch) : 노드를 나누는 기준  
깊이(depth) : 뿌리 노드와 잎 노드 까지의 노드 갯수

<img src="./image/27.png">

### 모델학습
#### 불순도
> `Decision Tree` 모델을 학습시키는 방법  
정보화 이론에서 사용하는 Gini 계수와 엔트로피를 사용한다.  
불순도가 0.5에 가까수록 불순도가 높고 0 혹은 1에 가까울 수록 순도가 높다.  
즉, 한 노드의 불순도가 가능한 많이 떨어지도록(순도가 올라가도록) 노드를 나눈다.

$$ Gini = 1 - \sum_1^n{(p_i)^2} $$

$$ Entropy = - \sum_1^n{p_iln(p_i)} $$

#### Gini index
위 예시에서 뿌리 노드 기준 지니계수 계산법  
class1 : 삼각형  
class2 : 동그라미  
>X < 0
>> True = class1 3개, class2 4개  
$1 - ({3 \over 3+4})^2 - ({4 \over 3+4})^2 = 0.48$  
False = class1 4개, class2 3개  
$1 - ({4 \over 4+3})^2 - ({3 \over 4+3})^2 = 0.48$  
total Gini 계수  
$1 - ({7 \over 7+7})0.48 - ({7 \over 7+7})0.48 = 0.52$

위 예시에서 잎 노드 기준 지니계수 계산법  
class1 : 삼각형  
class2 : 동그라미  
>Y < 1
>> True = class1 3개, class2 0개  
$1 - ({3 \over 3})^2 - ({0 \over 3})^2 = 0$  
False = class1 0개, class2 4개  
$1 - ({0 \over 4})^2 - ({4 \over 4})^2 = 0$  
total Gini 계수  
$1 - ({3 \over 3+4})0 - ({4 \over 3+4})0 = 1$

위의 예시에서 계산한 total Gini 계수가 곧 Decision tree 모델의 비용함수가 된다.  
이를 바탕으로 더 나은 선택을 하게 되는 결정경계를 생성하는 방법으로 데이터를 학습하는데 이를 greedy 알고리즘이라 한다.

### Decision Tree classifier 실습

In [ ]:
# 필요모듈 import
from sklearn.datasets import load_iris

In [ ]:
# iris 데이터로드
iris = load_iris()

In [ ]:
print(iris['DESCR'])

In [ ]:
import pandas as pd

In [ ]:
# 로딩 데이터 확인
X = pd.DataFrame(iris['data'], columns=iris['feature_names'])

In [ ]:
# 타겟데이터 분할
y = iris['target']

In [ ]:
# 학습 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# 모델 import
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# 모델 생성
dtc = DecisionTreeClassifier(random_state=42, max_depth=2)

In [ ]:
# 모델 학습
dtc.fit(X_train, y_train)

In [ ]:
# 모델평가
dtc_pred = dtc.predict(X_test)

In [ ]:
# test셋 분류 결과 확인
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, dtc_pred))
print(classification_report(y_test, dtc_pred, target_names=iris['target_names']))

In [ ]:
X_test

In [ ]:
# test data 시각화
import seaborn as sns
plt.figure(figsize=(6, 4))
sns.scatterplot(X_train, x='petal length (cm)', y='petal width (cm)', hue=y_train)
plt.vlines(2.45, 0, 2.5)
plt.hlines(1.55, 1, 6.5)
plt.show()

### 모델 해석을 위한 시각화 방법
#### feature importance
트리 기반 모델은 트리를 분기하는 과정에서 어떤 변수가 모델을 학습하는데 중요한지 살펴볼 수 있다.

In [ ]:
dtc.feature_importances_

In [ ]:
# feature importance 시각화
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 4))
plt.bar(X_train.columns, dtc.feature_importances_)

#### model plotting

In [ ]:
# 모델 시각화
from sklearn.tree import plot_tree
plt.figure(figsize=(10, 10))
plot_tree(dtc, 
          feature_names=iris['feature_names'], 
          class_names=iris['target_names'],
          filled=True)

### 가지치기 (pruning)
>`Decision Tree`모델은 모든 **잎 노드의 불순도가 0이 되는 순간까지 모델을 성장**시키면서 크기를 키워나간다.  
순수 노드로만 이루어진 트리 모델은 훈련 데이터를 100% 정확도로 맞출 수 있다.  
이러한 특성 때문에 트리 모델은 **과적합에 취약**하다.  
과적합 방지를 위해서는 **트리의 복잡도를 제어** 할 필요가 있다.

>과적합 방지를 위한 모델링 파라메터  
>> - **`max_depth`** : 트리의 최대 깊이  
- `max_leaf_nodes` : 잎 노드의 최대개수  
- `min_sample_leaf` : 잎 노드가 되기 위한 최소 샘플 갯수  
- `min_sample_split` : 잎 노드가 분지 되기 위한 최소 샘플 갯수

위의 iris 데이터는 3개의 클래스로 이루어진 데이터셋이지만 모델플로팅 결과 2뎁스의 노드에서 어느정도 데이터 구분이 되었습니다.  
이를 기준으로 사후 가지치기를 진행 해 보겠습니다.

## Decision tree regressor
> `Decision Tree`모델은 알고리즘 특성으로 분류 및 예측 모델링에 모두 사용이 가능하다.  
일반적으로 잎 노드에 속한 학습샘플의 값의 평균을 바탕으로 예측값을 결정한다.  
회귀모델 평가 방법인 MSE를 각 노드에 속한 샘플에 적용하고 이를 최소화 시킨다.  

<img src="./image/28.png">
<img src="./image/29.png">

### Decision tree regressor 실습

In [ ]:
# 보스턴 집값 데이터 로딩
df = pd.read_csv('./data/boston.csv')

In [ ]:
# 타겟 데이터 분할
y = df['y']
X = df.drop('y', axis=1)

In [ ]:
# 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 모델 import
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# 모델 생성
dtr = DecisionTreeRegressor(random_state=42, max_depth=12)
dtr.fit(X_train, y_train)

In [ ]:
# 모델 예측
dtr_pred = dtr.predict(X_test)

In [ ]:
# 모델 평가지표 출력
from sklearn.metrics import r2_score, mean_squared_error
print(r2_score(y_test, dtr_pred))
print(mean_squared_error(y_test, dtr_pred))

In [ ]:
'''
0.8444833592340152
11.588026315789474
'''

In [ ]:
# 모델 구조 출력
plt.figure(figsize=(10, 10))
plot_tree(dtr);